In [4]:
import pickle
import CO2_Dataset_Preparation
import CO2_functions
import CO2_Processing
from CO2_Dataset_Preparation import *
from CO2_functions import * 
from CO2_Processing import *

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib.dates as mdates
import matplotlib
from matplotlib import gridspec
import pickle

import importlib
importlib.reload(CO2_functions)
importlib.reload(CO2_Processing)
importlib.reload(CO2_Dataset_Preparation)

from CO2_Dataset_Preparation import *
from CO2_functions import *
from CO2_Processing import *

%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
idx = pd.date_range(start = '2019-08-15 00:00:00',end = '2019-11-27 23:00:00',freq ='S')
alldf = pd.DataFrame({'dt':idx})
alldf.set_index('dt',inplace  =True)

In [ ]:
tower = 'Picarro'
picarro_df = pd.DataFrame()
for i in range(1,7):
    position_number = i
    excess_rolls_sec = [60,600,3600]
    downsample_sec = 1

    data = Processed_Set(tower,position_number,excess_rolls_sec,vent_bool = True,wbb_bool=True)
    data._retrieve_data('../CO2_Data_Processed/')
    data._apply_excess(delete_min_cols=False)
    
    picarro_df = pd.concat([picarro_df,data.data['Picarro'].resample('1S').mean()],axis = 0)

Applying excess using minimum on windows: [60, 600, 3600] seconds
Applying excess using minimum on windows: [60, 600, 3600] seconds
Applying excess using minimum on windows: [60, 600, 3600] seconds
Applying excess using minimum on windows: [60, 600, 3600] seconds
Applying excess using minimum on windows: [60, 600, 3600] seconds
Applying excess using minimum on windows: [60, 600, 3600] seconds


In [ ]:
tower = 'Multi'
multi_df = pd.DataFrame()
wbb_df = pd.DataFrame()
for i in range(1,4):
    position_number = i
    excess_rolls_sec = [60,600,3600]
    downsample_sec = 1

    data = Processed_Set(tower,position_number,excess_rolls_sec,vent_bool = True,wbb_bool=True)
    data._retrieve_data('../CO2_Data_Processed/')
    data._apply_excess(delete_min_cols=False)
    
    multi_df = pd.concat([multi_df,data.data['Multi']],axis = 0)
    wbb_df = pd.concat([wbb_df,data.data['Vent_Mass']],axis = 0)
multi_df.sort_index(axis = 'index',inplace = True)
wbb_df.sort_index(axis = 'index',inplace = True)
wbb_df.drop(['Rotations','Q'],axis = 1,inplace = True)
wbb_cols = ['Vent_Raw_Conc','Vent_Exit_Vel','Vent_Inner_Temp','Vent_Outer_Temp','WBB_CO2','WBB_CH4','Vent_Excess_Conc','Vent_Mass_Flow']
wbb_df.columns = wbb_cols
multi_df.rename(columns = {'Temp':'Multi_Temp','ws' : 'Multi_WS','wd':'Multi_WD'},inplace = True)

Applying excess using minimum on windows: [60, 600, 3600] seconds
Applying excess using minimum on windows: [60, 600, 3600] seconds
Applying excess using minimum on windows: [60, 600, 3600] seconds


In [ ]:
alldf = pd.concat([alldf,picarro_df,multi_df,wbb_df],axis=1).reindex(alldf.index)

In [ ]:
pickle_out = open('../CO2_Data_Processed/all_df_concatenated.pkl','wb')
pickle.dump(alldf,pickle_out)
pickle_out.close()

In [5]:
pickle_in = open('../CO2_Data_Processed/all_df_concatenated.pkl','rb')
alldf = pickle.load(pickle_in)

In [6]:
alldf

,Pic_CO2,Pic_CH4,Pic_Loc,ANEM_Y,ANEM_X,ANEM_Z,min_r60_Pic_CO2,min_r60_Pic_CH4,min_r600_Pic_CO2,min_r600_Pic_CH4,...,excess_r3600_CO2_2,excess_r3600_CO2_3,Vent_Raw_Conc,Vent_Exit_Vel,Vent_Inner_Temp,Vent_Outer_Temp,WBB_CO2,WBB_CH4,Vent_Excess_Conc,Vent_Mass_Flow
dt,,,,,,,,,,,,,,,,,,,,,
2019-08-15 00:00:00,413.894950,1.722919,1.0,-0.046262,-0.315575,0.038520,413.24851,1.715718,412.854368,1.714603,...,NaN,NaN,NaN,NaN,NaN,NaN,416.6410,1.916160,NaN,NaN
2019-08-15 00:00:01,413.815115,1.723759,1.0,-0.066340,-0.344498,-0.039098,413.24851,1.715704,412.854028,1.714603,...,NaN,NaN,NaN,NaN,NaN,NaN,416.6063,1.916039,NaN,NaN
2019-08-15 00:00:02,413.766612,1.724143,1.0,-0.062104,-0.372218,-0.074662,413.24851,1.715691,412.853690,1.714603,...,NaN,NaN,NaN,NaN,NaN,NaN,416.5716,1.915918,NaN,NaN
2019-08-15 00:00:03,413.956135,1.723348,1.0,-0.164199,-0.436733,-0.151772,413.24851,1.715679,412.853354,1.714603,...,NaN,NaN,NaN,NaN,NaN,NaN,416.5369,1.915797,NaN,NaN
2019-08-15 00:00:04,414.041693,1.725913,1.0,-0.172617,-0.431376,-0.141552,413.24851,1.715668,412.853021,1.714603,...,NaN,NaN,NaN,NaN,NaN,NaN,416.5022,1.915676,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-27 22:59:56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,457.8376,2.102800,NaN,NaN
2019-11-27 22:59:57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,457.7877,2.102725,NaN,NaN
2019-11-27 22:59:58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,457.7378,2.102650,NaN,NaN
